In [218]:
import numpy as np
import pandas as pd 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import os
from sklearn import metrics
from unicodedata import normalize
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rodrigo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [163]:
imdb_dir = 'aclImdb/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [164]:
test_dir = os.path.join(imdb_dir, 'test')
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

# Questão 1

In [165]:
def removerCaracteresEspeciais(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text


In [166]:
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [167]:
stop = stopwords.words('english')
def removeStopWords(texto):
    texto = removerCaracteresEspeciais(texto)
    aux = [w for w in tokenizer_porter(texto) if w not in stop]
    output = ''
    output = output + aux[0]
    for i in aux[1:]:
        output = output + ' ' + i
    return output

In [168]:
for i in range(0,len(texts)):
    texts[i] = removeStopWords(texts[i])

In [169]:
texts[0]

'idea wa awesom actor incred stori could veri scari write wa poor wa depth realli get thi movi feel charact lot cliffhang movi end veri weirdli wa happi end know wa sad end know leav theater feel unsatisfi movi much give becaus edit mean right realli recommend thi movi becaus say left movi feel like wa complet confus trust probabl thank watch thi movi 3 10'

# Questão 2

In [170]:
X_train = texts[:40000]
y_train = labels[:40000]
X_test = texts[:40000]
y_test = labels[:40000]

In [178]:
X_train_tf.shape

(40000, 65969)

In [314]:
count_vect = CountVectorizer(vocabulary='vocabulary')

In [186]:
X_train_c = count_vect.fit_transform(X_train)
X_test_c = count_vect.fit_transform(X_test)

In [187]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_c)
X_train_tf = tf_transformer.transform(X_train_c)

In [188]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_test_c)
X_test_tf = tf_transformer.transform(X_test_c)

In [189]:
mNB = MultinomialNB().fit(X_train_tf, y_train)

In [191]:
y_pred = mNB.predict(X_train_tf)

In [203]:
metrics.accuracy_score(y_test,y_pred)

0.710925

# Questão 3

In [205]:
mSGD = SGDClassifier()

In [248]:
vect = HashingVectorizer(decode_error='ignore', n_features=2**21)

In [ ]:
X = vect.fit_transform(X_train)
mSGD.fit(X, y_train);++

In [210]:
y_pred_sgd = mSGD.predict(X)

In [211]:
metrics.accuracy_score(y_test,y_pred_sgd)

0.901275

# Questão 4

In [213]:
bern = BernoulliNB()

In [215]:
bern.fit(X_train_tf, y_train);

In [216]:
y_pred = bern.predict(X_test_tf)

In [217]:
metrics.accuracy_score(y_test, y_pred)

0.873

In [249]:
gnb = GaussianNB()

In [298]:
class DenseTransformer(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [329]:
pipeline = Pipeline([('bow',CountVectorizer()),
                           ('to_dense', DenseTransformer()), 
                           ('classifier',GaussianNB())])

In [ ]:
pipeline.predict(X_test)